In [57]:
# for reading in data
import pandas as pd

# data preprocessing
from pvops.text.preprocess import text_remove_numbers_stopwords
from nltk import corpus

# machine learning pipeline: vectorizer, model definitions, and scoring
from pvops.text.classify import classification_deployer
from pvops.text.defaults import supervised_classifier_defs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, make_scorer


### Read in data

First, we'll read in the data and take a look at it. We would like to identify inverter-related records, so our column of interest is `Asset`.

In [58]:
CM_df = pd.read_csv('example_data/example_CM_records.csv')
CM_df['Asset'].head(10)

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
Name: Asset, dtype: object

We can see that many records have a missing `Asset` field. Let's see how many records are missing it.

In [59]:
count_null = CM_df['Asset'].isnull().sum()
print(f'{count_null}/{len(CM_df)} records have a missing asset ({count_null/len(CM_df)*100 :.1f}%).')

66/1093 records have a missing asset (6.0%).


We would like to leverage as much of the data as we can for later analysis, so it would help to find a way to gap-fill these records that are missing the asset. We can do so by leveraging the plain-text event description in the `GeneralDesc` column. Let's pull out the data of interest.

In [60]:
# pull out the Asset and GeneralDesc columns, and any rows with nulls in either column
CM_nonnull = CM_df[['Asset','GeneralDesc']].dropna()
CM_nonnull.head(5)

,Asset,GeneralDesc
20,Inverter,Inverter underperforming due to unknown
21,Inverter,Inverter not producing due to Unknown
22,Inverter,Inverter Faulted Due Hardware Malfunction
24,Tracking System,Tracker repair.
27,Tracking System,Motor 1 registering encoder feedback error.. E...


For good measure, we can also look to see how many of the non-null records have an inverter-related asset:

In [61]:
count_inverter = CM_nonnull['Asset'].str.contains('Inverter').sum()
print(f'{count_inverter}/{len(CM_nonnull)} records have an inverter-related asset ({count_inverter/len(CM_nonnull)*100 :.1f}%).')

384/1027 records have an inverter-related asset (37.4%).


### Naive approach: keyword method

As a first approach, we can perform a simple keyword-based method. We make a prediction of the asset based on whether the word "inverter" is present in the description.

In [62]:
# check if the event description contains the word 'inverter'; this is our prediction
prediction = CM_nonnull['GeneralDesc'].str.lower().str.contains('inverter')
# check if the asset itself contains the word 'inverter'; this is the true label
true_label = CM_nonnull['Asset'].str.lower().str.contains('inverter')

naive_num_correct = (prediction == true_label).sum()
print(f'{naive_num_correct}/{len(CM_nonnull)} records predicted correctly from keyword search '
      + f'({naive_num_correct/len(CM_nonnull)*100 :.1f}%).')

835/1027 records predicted correctly from keyword search (81.3%).


This already does decently well. Let's see if we can improve our results via a more robust machine learning approach.

### Prepare data for supervised classification

First, we need to clean our text data. We can use `nltk`'s english stopwords list to remove common words that won't have much bearing on the true content of the records. All we need to do is pass this into the `text_remove_numbers_stopwords()` function from `pvops.text.preprocess`, which will handle all of this for us.

From there, we can make everything lowercase, and we're basically done. 

In [63]:
# clean the event description
stopwords = corpus.stopwords.words('english')
CM_nonnull['GeneralDescCleaned'] = CM_nonnull.apply(lambda x: text_remove_numbers_stopwords(x['GeneralDesc'], stopwords),
                                                    axis='columns')

# set event description to be lowercase
CM_nonnull['GeneralDescLower'] = CM_nonnull['GeneralDescCleaned'].str.lower()

CM_nonnull.head(5)

,Asset,GeneralDesc,GeneralDescCleaned,GeneralDescLower
20,Inverter,Inverter underperforming due to unknown,Inverter underperforming due unknown,inverter underperforming due unknown
21,Inverter,Inverter not producing due to Unknown,Inverter producing due Unknown,inverter producing due unknown
22,Inverter,Inverter Faulted Due Hardware Malfunction,Inverter Faulted Due Hardware Malfunction,inverter faulted due hardware malfunction
24,Tracking System,Tracker repair.,Tracker repair,tracker repair
27,Tracking System,Motor 1 registering encoder feedback error.. E...,Motor registering encoder feedback error Encod...,motor registering encoder feedback error encod...


Finally, we would like to have our response be a binary value: whether the asset relates to inverters or not. We can use the `remappings_asset.csv` file to help us with this.

In [64]:
remappings_df = pd.read_csv('example_data/remappings_asset.csv')
remappings_df[-10:] # end of the dataframe is more clear on what it is doing

,in,out_
44,Point of Interconnection,other
45,Racking/Trackers,tracker
46,Rooftop PV System,other
47,Site,other
48,String,other
49,String Inverter,inverter
50,Subarray,other
51,Summary,other
52,Tracker control unit,tracker
53,Tracking System,tracker


All we need to do is put together a dictionary that will let us map the asset in our dataframe to a more general asset description.

In [65]:
remapping_dict = {row['in'].lower() : row['out_'].lower() for _, row in remappings_df.iterrows()}
CM_nonnull['SimpleAsset'] = CM_nonnull['Asset'].apply(lambda x : remapping_dict[x.lower()])

# define our x as the general event description, and y as whether the asset is related to inverters
x = CM_nonnull['GeneralDescLower']
y = CM_nonnull['SimpleAsset'] == 'inverter'

### Model training and selection

Now, it's time to begin the model training. First, we need to turn our `x` into numerical data, which we can do via tf-idf vectorization. We'd like our vectorization to include the entire corpus of our event descriptions, even those where the asset is null. This is so we can use our exact same pipeline to predict the missing asset labels in the final step.

In [66]:
# fit the tf-idf vectorizer on the whole corpus, including the rows w/o asset that we plan to gapfill using our model
all_descr = CM_df.dropna(subset='GeneralDesc')['GeneralDesc']
all_descr = all_descr.str.lower().astype('str')
vectorizer = TfidfVectorizer(min_df=1, stop_words=stopwords, ngram_range=(1,2), sublinear_tf=True)
vectorizer.fit(all_descr);

Now, we can fit models across various parameters using the `classification_deployer()` function from `pvops.text.classify`. All we need to do is define our pipeline, our models, our parameters, and our scoring function.

The pipeline and scoring function are easy enough. We can get a good set of model parameters to search over from `pvops.text.defaults`, as shown below:

In [67]:
# first do vectorization, then classification
pipeline_steps = [('tfidf', vectorizer),
                  ('clf', None)]

# basic accuracy score metric
scorer = make_scorer(accuracy_score)

# models that we will use
model_names = ['LinearSVC', 'LogisticRegression', 'PassiveAggressiveClassifier']

# get the default parameter grid from pvops for these models
(params, model_instances) = supervised_classifier_defs('detailed')
classifiers = {model_name : model_instances[model_name] for model_name in model_names}
param_grid = {model_name : params[model_name] for model_name in model_names}

We can see below what our resulting `classifiers` and `param_grid` variables look like:

In [68]:
print(classifiers)
param_grid

{'LinearSVC': LinearSVC(), 'LogisticRegression': LogisticRegression(), 'PassiveAggressiveClassifier': PassiveAggressiveClassifier()}


{'LinearSVC': {'clf__C': [0.01, 0.1, 1, 10.0, 100.0, 1000.0],
  'clf__max_iter': [800, 1000, 1200, 1500, 2000]},
 'LogisticRegression': {'clf__solver': ['newton-cg', 'lbfgs', 'sag'],
  'clf__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
         5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
         3.59381366e+03, 1.00000000e+04])},
 'PassiveAggressiveClassifier': {'clf__C': [0.001, 0.01, 0.1, 1.0],
  'clf__loss': ['hinge', 'squared_hinge']}}

Finally, all we need to do is pass everything into `classification_deployer()`:

In [69]:
summary_table, best_model = classification_deployer(x,
                                                    y,
                                                    n_splits=5,
                                                    classifiers=classifiers,
                                                    search_space=param_grid,
                                                    pipeline_steps=pipeline_steps,
                                                    scoring=scorer)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\agmoore\AppData\Local\anaconda3\envs\pvops_bare_with_tweaks\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Better score (0.891) found on classifier: LinearSVC
Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\agmoore\AppData\Local\anaconda3\envs\pvops_bare_with_tweaks\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Better score (0.897) found on classifier: LogisticRegression
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Better score (0.898) found on classifier: PassiveAggressiveClassifier


And below, we can see a summary of how the models performed.

In [71]:
sorted_table = summary_table.sort_values('mean_score').iloc[::-1].head(10)
sorted_table

,clf__C,clf__max_iter,mean_fit_time,estimator,min_score,max_score,mean_score,std_score,clf__solver,clf__loss
63,0.01,NaN,0.259973,PassiveAggressiveClassifier,0.765854,0.956098,0.897703,0.068552,NaN,squared_hinge
56,3593.813664,NaN,0.366188,LogisticRegression,0.760976,0.956098,0.896732,0.070559,sag,NaN
62,0.01,NaN,0.304948,PassiveAggressiveClassifier,0.760976,0.956098,0.893815,0.070214,NaN,hinge
50,464.158883,NaN,0.486903,LogisticRegression,0.765854,0.95122,0.892839,0.066008,sag,NaN
53,1291.549665,NaN,0.412428,LogisticRegression,0.756098,0.95122,0.892834,0.070207,sag,NaN
66,1.0,NaN,0.254419,PassiveAggressiveClassifier,0.760976,0.95122,0.891849,0.069144,NaN,hinge
59,10000.0,NaN,0.384808,LogisticRegression,0.765854,0.95122,0.890893,0.065184,sag,NaN
24,100.0,2000,0.584124,LinearSVC,0.770732,0.946341,0.890888,0.062957,NaN,NaN
22,100.0,1200,0.514852,LinearSVC,0.770732,0.946341,0.890888,0.062957,NaN,NaN
21,100.0,1000,0.422944,LinearSVC,0.770732,0.946341,0.890888,0.062957,NaN,NaN


So we got that a `PassiveAggressiveClassifier` did the best, although many models seemed to have performed about the same, around 90% accuracy. It could be argued then that the increased complexity of a `PassiveAggressiveClassifier` isn't worth the very small model improvement over the simpler `LogisticRegression`.

Note that our best-performing models did perform notably better than our naive approach, so we do gain something by using machine learning methods.

`classification_deployer()` only returns the best model, but we can easily refit using just the parameters we want.

In [72]:
# get the row for the model we want to use
model_row = sorted_table.iloc[1,:]

# pull out the model name and parameters
model_name = model_row['estimator']
model_params = model_row[list(param_grid[model_name].keys())].to_dict()

# parameter values have to be in a list, even if there is just one
model_params = {param_name: [param_val] for param_name, param_val in model_params.items()}

# get into the form used by classification_deployer()
single_classifier = {model_name : model_instances[model_name]}
single_param_grid = {model_name : model_params}
single_classifier, single_param_grid

({'LogisticRegression': LogisticRegression()},
 {'LogisticRegression': {'clf__solver': ['sag'],
   'clf__C': [3593.813663804626]}})

Now that we have what we need, we can quickly rerun the fit and save the final model.

In [73]:
_, final_model = classification_deployer(x, y,
                                            n_splits=5,
                                            classifiers=single_classifier,
                                            search_space=single_param_grid,
                                            pipeline_steps=pipeline_steps,
                                            scoring=scorer)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Better score (0.895) found on classifier: LogisticRegression


c:\Users\agmoore\AppData\Local\anaconda3\envs\pvops_bare_with_tweaks\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### Gap-filling the asset using our final model

Now, we can use our model to make predictions on the rows with a missing `Asset` (but with a `GeneralDesc`). First, we can pull out the data we need to feed into the model.

In [74]:
records_to_gapfill = CM_df[CM_df['Asset'].isnull()
                           & ~CM_df['GeneralDesc'].isnull()].copy()
# only need to keep around the GeneralDesc field
records_to_gapfill = records_to_gapfill['GeneralDesc']

Now, we follow the same cleaning procedure as before, and finally use our model to predict the asset.

In [75]:
records_to_gapfill = records_to_gapfill.apply(lambda x: text_remove_numbers_stopwords(x, stopwords))
records_to_gapfill = records_to_gapfill.str.lower()
pred_is_inverter = final_model.predict(records_to_gapfill)

Now, we can use our prediction as a mask to pull out a list of indices where the asset is predicted to be an inverter.

In [76]:
pred_inverter_indices = records_to_gapfill.index[pred_is_inverter.astype(bool)]
pred_inverter_indices

Index([  1,   4,   5,   6,   7,   9,  10,  12,  14,  15,  17,  19,  60,  98,
       109, 135, 137, 168, 170, 177, 179, 180, 185, 188, 199, 200, 202, 203],
      dtype='int64')

We can also construct a mask for the datapoints without null values for the asset. By combining these two masks, we get a final set of inverter-related records that can be used for other analysis.

In [77]:
inverter_indices = CM_df.index[~CM_df['Asset'].isnull()
                               & CM_df['Asset'].str.contains('Inverter')]

inverter_CM_df = CM_df.loc[list(inverter_indices) + list(pred_inverter_indices), :]
inverter_CM_df.loc[pred_inverter_indices, 'Asset'] = 'Inverter'
inverter_CM_df

,randid,Date_EventStart_Consistent,Date_EventEnd_Consistent,GeneralDesc,CompletionDesc,Asset,states,Commissioning_Date,DC_Size_kW,AC_Size_kW,PlantType,ArrayType,InverterType,InverterOEM
20,C2S54,2018-03-16 11:55:00,2018-03-29 13:40:00,Inverter underperforming due to unknown,Power Cycle. Hardware Replaced:,Inverter,CA,2012-07-23,181.0,250.0,Distributed Generation,NaN,central,NaN
21,C2S54,2017-09-22 14:40:00,2017-09-26 10:30:00,Inverter not producing due to Unknown,Field Wiring Repair. Hardware Replaced:,Inverter,CA,2012-07-23,181.0,250.0,Distributed Generation,NaN,central,NaN
22,C2S54,2017-02-01 08:00:00,2017-02-10 14:25:00,Inverter Faulted Due Hardware Malfunction,Power Cycle. Hardware Replaced:,Inverter,CA,2012-07-23,181.0,250.0,Distributed Generation,NaN,central,NaN
32,C3S300,2017-07-25 09:00:00,2017-07-25 15:01:00,All inverters not communcating but still produ...,SOLV remotely fixed comms.,Central Inverter,OR,2016-12-03,5891.0,6600.0,Utility-scale,Tracker,NaN,SMA America
42,C3S300,2018-06-04 09:00:00,2018-06-05 00:45:00,WO created for support on retrofit for SMA inv...,Displays replaced on all inverters on site.,Central Inverter,OR,2016-12-03,5891.0,6600.0,Utility-scale,Tracker,NaN,SMA America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,C3S211,2017-03-29 09:00:00,2017-03-29 17:14:00,"Site comms down. Modem is reachable, datalogge...",troubleshot site comm issue.,Inverter,NC,2015-12-10,6326.0,5550.0,Utility-scale,Fixed,central,SMA America
199,C3S80,2017-06-07 09:00:00,2017-06-27 16:10:00,All inverters not producing. B Voltage spike b...,DSI Replaced the phase B dip pole fuse. No CCR...,Inverter,NC,2016-09-20,7008.0,5280.0,Utility-scale,Fixed,central,SMA America
200,C3S80,2016-11-14 09:00:00,NaN,11/04/16 Maxton Inverter-2.3 Low on Production...,Reconfigured inv 1's ip. Checked string perfor...,Inverter,NC,2016-09-20,7008.0,5280.0,Utility-scale,Fixed,central,SMA America
202,C3S80,2017-01-06 09:00:00,2017-02-13 12:05:00,MEC is at Maxton working on labeling the inver...,.,Inverter,NC,2016-09-20,7008.0,5280.0,Utility-scale,Fixed,central,SMA America


In [78]:
inverter_CM_df.to_csv('example_data/example_CM_inverter_records.csv', index=False)